# Evaluation

In [1]:
import sys
import os

repo_path = os.path.sep.join(os.getcwd().split(os.path.sep)[:-3])
print(repo_path)
sys.path.append(repo_path)

/home/loebbert/projects/deepclustering/LMU_Master_Practical_SoSe24


In [3]:
from clustpy.deep.autoencoders import FeedforwardAutoencoder
from practical.DeepClustering.DeepECT.evaluation_pipeline import DatasetType, evaluate

## Flat Clustering

### MNIST

In [4]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.MNIST, seed=42
)

### USPS

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.USPS, seed=42
)

### Fashion-MNIST

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.FASHION_MNIST, seed=42
)

### Reuters

In [ ]:
evaluate(
    init_autoencoder=FeedforwardAutoencoder, dataset_type=DatasetType.REUTERS, seed=42
)

## Hierarchical Clustering

### MNIST

### USPS

### Fashion-MNIST

### Reuters